In [3]:
from MySQL_DB_connect_functions import *
import numpy as np

In [4]:
host_name, dbname, schema_name, port, username, password = get_db_info()

In [5]:
cnx = None

In [6]:
cnx = connect_to_db(username, password, host_name, schema_name, port)

Connected to Database


In [7]:
cursor = cnx.cursor()

In [8]:
# Name of schema/table storing requested data in the database
table_name = 'videos'

In [9]:
analysis_df = return_df_from_db(table_name, cnx)

In [10]:
analysis_df.head()

,video_id,channelTitle,title,description,publishedAt,viewCount,likeCount,favoriteCount,commentCount,caption,publishDayName,durationSecs,tagCount
0,_UoTTq651dE,3Blue1Brown,Why 5/3 is a fundamental constant for turbulence,Some mathematical order amidst the chaos of tu...,2018-11-07,622119,23357,0,822,1,Wednesday,688,10
1,-9OUyo8NFZg,3Blue1Brown,Euler's Formula and Graph Duality,"A description of planar graph duality, and how...",2015-06-21,412304,9630,0,331,1,Sunday,447,12
2,-qgreAUpPwM,3Blue1Brown,Pure Fourier series animation montage,Because why not?\nLearn the math behind this: ...,2019-07-03,600345,18533,0,1241,1,Wednesday,749,8
3,-RdOwhmqP5s,3Blue1Brown,"Newton’s method produces this fractal, why don...",Who knew root-finding could be so complicated?...,2021-10-12,1832565,74928,0,3107,1,Tuesday,1566,7
4,1SMmc9gQmHQ,3Blue1Brown,How to count to 1000 on two hands,Typically when we think of counting on two han...,2015-09-18,385566,11316,0,1087,1,Friday,154,11
